# FHIR Search API Exploration

Evaluate the search features on a handful of FHIR servers. This demonstrates a bunch of search queries on the servers being evaluated. Scroll down past the evaluation section to run the notebook and see it in action. 

## Summary

Evaluations are listed from least promising to most promising

### Microsoft FHIR Server on Azure

Pros
- Open source
- Free
- A "straight" FHIR server that adheres to FHIR spec
- Good error handling and reporting
    - Reports when a search param or operation is not supported

Cons
- Only runs on Azure
- Very little documentation
- Does not support a lot of major search functionality
    - Doesn't support full text search
    - Doesn't support custom queries
    - Chaining/reverse chaining
    - Including references


### Vonk

Pros
- A "straight" FHIR server that adheres to FHIR spec
- Has ~70% of search API spec implemented
- Decent documentation
- Vonk team is fairly responsive to issue reports (0.5 - 1day) on Zulip

Cons
- Not open source
- Not free
- Production releases are very buggy
- Not as quick to turn around fixes and releases as Aidbox
- Ignores search parameters it doesn't understand and returns everything
- Error handling and reporting is confusing or lacking
- Has a limited policy engine for access control
- Doesn't support full text search
- Doesn't support custom queries


### Smile CDR (commercial HAPI)

Pros
- A "straight" FHIR server that adheres to FHIR spec
- Has 70% of search API spec implemented
- Looks like professional/detailed documentation
- Looks like there are a lot of features - haven't had time to delve into them all
- Built on top of HAPI, open source FHIR server
- Good error handling and reporting
    - Reports when a search param or operation is not supported
- Supports full text search
- Supports multitenancy and seems to have a lot of features relating to auth

Cons
- Not free
- Not sure if it supports some of the basic search parameters

### Aidbox

Pros
- Has 70% of search API spec implemented
- Supports special search features that make it very flexible
    - Full text search
    - Query by SQL
    - Save SQL queries as endpoints
    - Query by exact path
- Decent documentation
- Team is very responsive to issues and makes releases often
- Good error handling and reporting
    - Reports when a search param or operation is not supported
- Aidbox representation of conformance resources is much easier to digest
  and understand than FHIR conformance resources
- Uses PostgresSQL as their database and its open source
- Has a GraphQL API - haven't tested it yet

Cons
- Represents conformance resources (e.g. extensions, search parameters in their own way
    - Until they've developed the tools to easily go between Aidbox and FHIR, we will 
      have to do this ourselves
- Does not seem to support the :missing: modifier for search

In [2]:
import requests
from click.testing import CliRunner
from pprint import pprint, pformat
import pandas

from requests.auth import HTTPBasicAuth

from kf_model_fhir.config import FHIR_VERSION, SERVER_CONFIG, PROJECT_DIR
from kf_model_fhir import cli
from kf_model_fhir.client import FhirApiClient
from kf_model_fhir.loader import load_resources
from kf_model_fhir.utils import read_json
from kf_model_fhir.aidbox import *

# Remove the local vonk server for now
# We have a public vonk server to use instead
SERVER_CONFIG.pop('vonk', None)
runner = CliRunner()

### Helper Functions ###
def run_cli_cmd(cmd_name, params_list):
    """
    Run a fhirmodel CLI command
    """
    cmd = getattr(cli, cmd_name)
    result = runner.invoke(cmd,params_list)
    if result.exit_code != 0:
        raise Exception(result.exception)
        

def emoji(resp):
    """
    Return an emoji based on the `total` value in a FHIR GET response
    """
    total = resp.get('total', None)
    if total is None:
        return '❌'
    if total > 0:
        return '✅'
    else:
        return '⚠️ or ✅'

def execute_queries(queries, display_content=False):
    """
    Run all queries in `queries` on every server
    Display results
    """
    for query in queries:
        dlen = len(query['desc'])
        print('*' * dlen)
        print(query['desc'])
        print('*' * dlen)
        for server_name, settings in SERVER_CONFIG.items():
            print(server_name.upper())
            print('-' * len(server_name))
            url = f"{settings['base_url']}/{query['endpoint']}"
            uname = settings.get('username')
            pw = settings.get('password')
            params = query['params']
            params.update({'_total': 'accurate'})
            resp, status_code = get(url, username=uname, password=pw, params=params)

            if display_content:
                pprint(resp)
                
            print(f"{emoji(resp)} Found {resp.get('total')} {query['endpoint'].lstrip('/')} "
                  f"matching these params {pformat(params)}")
            if resp.get('total') == counts[query['endpoint']]:
                print(
                    f'⚠️ ？ The returned total = actual count of {query["endpoint"]}. '
                    f'This may or may not be correct. Some servers ignore search parameters '
                    'that they do not understand, which results in a get all query. '
                )
            print('\n')
print(f'Servers being evaluated: {pformat(list(SERVER_CONFIG.keys()))}')

Servers being evaluated: ['aidbox', 'smile-cdr', 'hapi', 'azure', 'vonk-kidsfirst-public']


## Setup Required

Every server being evaluated is publically hosted so you don't need to spin up any docker containers. You just need to clone the `kf-model-fhir` repo and switch to the `search-api-testing` branch

### 1. Get the Code

```shell
# Get code
git clone git@github.com:kids-first/kf-model-fhir.git
cd kf-model-fhir

# Switch to right branch
git checkout search-api-testing
```

### 2. Setup Virtual Environment

```shell
# Setup virtual env
python3 -m venv venv
source venv/bin/activate

# Install requirements
pip install -e .
```

Now you're ready to run this notebook

## Important Notes

### * Your Network Might Block Some FHIR Servers
For us at chop, this means you have to be on `chopguest` to run this since `chopnet` blocks the Smile CDR server

### * Disclaimer - Throw Away Code
Code in this branch is throw away code and only meant for search API exploration - don't judge :)
There are probably bugs and things might break if you change certain things

## Generate the Test Data

In [ ]:
# Generate resources
run_cli_cmd('generate', ['./resources'])

## Test Data Description

- Data for this notebook is in the `kf-model-fhir/project` folder. 
- Conformance resources like StructureDefinitions and SearchParameters are in `kf-model-fhir/project/profiles` 
- Dummy resources that were generated from the step earlier are located `kf-model-fhir/project/resources`

In [3]:
resources = load_resources(os.path.join(PROJECT_DIR, 'resources'))
df = pandas.DataFrame(
    [
        {'resource_type': r['resource_type'],
         'id': r['content'].get('id'),
         'references': r['content'].get('subject', {}).get('reference'),
        }
        for r in resources
    ]
)
display(df)
counts = df.groupby(['resource_type']).size()
display(counts)

2019-11-14 15:09:53,333 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Condition-CD-00000-0.json
2019-11-14 15:09:53,336 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Condition-CD-00001-0.json
2019-11-14 15:09:53,337 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Condition-CD-00002-0.json
2019-11-14 15:09:53,342 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Condition-CD-00003-0.json
2019-11-14 15:09:53,343 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Condition-CD-00004-0.json
2019-11-14 15:09:53,344 - kf_model_fhir.loader - DEBUG - Reading resource file: /User

2019-11-14 15:09:53,436 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Specimen-BS-00007-1.json
2019-11-14 15:09:53,437 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Specimen-BS-00008-0.json
2019-11-14 15:09:53,438 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Specimen-BS-00008-1.json
2019-11-14 15:09:53,440 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Specimen-BS-00009-0.json
2019-11-14 15:09:53,441 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/pull-request-reviews/kf-model-fhir/project/resources/Specimen-BS-00009-1.json


,resource_type,id,references
0,Condition,CD-00000-0,Patient/PT-00000
1,Condition,CD-00001-0,Patient/PT-00001
2,Condition,CD-00002-0,Patient/PT-00002
3,Condition,CD-00003-0,Patient/PT-00003
4,Condition,CD-00004-0,Patient/PT-00004
5,Condition,CD-00005-0,Patient/PT-00005
6,Condition,CD-00006-0,Patient/PT-00006
7,Condition,CD-00007-0,Patient/PT-00007
8,Condition,CD-00008-0,Patient/PT-00008
9,Condition,CD-00009-0,Patient/PT-00009


resource_type
Condition      10
Observation    10
Patient        10
Specimen       20
dtype: int64

## Load the Test Data

This might take a couple minutes. Our test data is deleted first from every server and than POST/PUT to every server. The servers are slow

### You might be able to skip this since the servers probably have data loaded

In [ ]:
# Publish profiles and resources to server
for server_name in SERVER_CONFIG:
    # Profiles
    server_settings = SERVER_CONFIG[server_name]
    print(f'\n\n********************** {server_name.upper()} **********************')
    username = server_settings.get('username')
    password = server_settings.get('password')    
    params = ['./profiles', '--resource_type', 'profile',
          '--server_name', server_name]
    if username and password:
        params.extend(['--username', username, '--password', password])    
    
    if server_name == 'aidbox':
        params.extend(['-e', 'search_parameter'])
        load_search_params(username, password)
        
    run_cli_cmd('publish', params)
    
    # Resources
    params = ['./resources', '--resource_type', 'resource',
          '--server_name', server_name]
    if username and password:
        params.extend(['--username', username, '--password', password])

    run_cli_cmd('publish', params)
    print(f'\n\n*******************************************************************')

## FHIR Search API Spec

https://www.hl7.org/fhir/search.html

## What are SearchParameters?

https://www.hl7.org/fhir/searchparameter.html

You might be wondering how you can do a query like this: `/Patient?family=Holmes` because `family` is a nested attribute in the Patient payload: `name: [ {family: Holmes} ]`. Well, this is where SearchParameters come in!

A SearchParameter is a conformance resource that defines what part of the resource payload to get the value of a search attribute. For example, in the SearchParameter definition for `family`, it probably has a path defined in it like this: `Patient.name.family`. Since `family` is part of the base FHIR Patient resource, this SearchParameter is probably already loaded into most FHIR servers that have the base FHIR resources.

If you really want to see what the SearchParameter looks like try this:

In [9]:
url = f"{SERVER_CONFIG['smile-cdr']['base_url']}/SearchParameter"
params = {'base': 'Patient', 'code': 'family'}
resp, status_code = get(url, params=params)
print('\nSearchParameter definition for `family` attribute on Patient resource\n')
pprint(resp['entry'][0]['resource'])

Success: GET https://try.smilecdr.com:8000/SearchParameter

SearchParameter definition for `family` attribute on Patient resource

{'base': ['Patient'],
 'code': 'family',
 'description': 'A portion of the family name of the patient',
 'expression': 'Patient.name.family',
 'id': '1234',
 'meta': {'lastUpdated': '2017-07-09T17:57:42.138+00:00', 'versionId': '1'},
 'resourceType': 'SearchParameter',
 'status': 'active',
 'type': 'string'}


## Basic Queries

In [5]:
queries = [
    {
        'desc': 'Get all patients',
        'endpoint': 'Patient',
        'params': {}
    },
    {
        'desc': 'Get all female patients',            
        'endpoint': 'Patient',
        'params': {'gender': 'female'}
    },
    {
        'desc': 'Get all female patients with last name = Holmes',        
        'endpoint': 'Patient',
        'params': {'gender': 'female', 'family': 'Holmes'}
    }
]
execute_queries(queries)

****************
Get all patients
****************
AIDBOX
------
Success: GET https://kidsfirstr4.aidbox.app/fhir/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The returned total = actual count of Patient. This may or may not be correct. Some servers ignore search parameters that they do not understand, which results in a get all query. 


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Patient
✅ Found 1008 Patient matching these params {'_total': 'accurate'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Patient
✅ Found 3652 Patient matching these params {'_total': 'accurate'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The returned total = actual count of Patient. This may or may not be correct. Some servers ignore search parameters that they do not understand, which results in a get all query. 


VONK-KIDSFIRST-PUBLIC
-----

## Using Modifiers

These are strings that start and/or end with `:` and get appended onto the name of the search parameter
you're using in a query.

https://www.hl7.org/fhir/search.html#modifiers

In [6]:
queries = [
    {
        'desc': 'Get all patients missing the gender attribute',
        'endpoint': 'Patient',
        'params': {'gender:missing': True}
    },
    {
        'desc': 'Get all patients that are NOT female',            
        'endpoint': 'Patient',
        'params': {'gender:not': 'female'}
    },
    {
        'desc': 'Get all female patients with name containing Hol',        
        'endpoint': 'Patient',
        'params': {'gender': 'female', 'name:contains': 'Hol'}
    },
]
execute_queries(queries)

*********************************************
Get all patients missing the gender attribute
*********************************************
AIDBOX
------
Success: GET https://kidsfirstr4.aidbox.app/fhir/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate', 'gender:missing': True}
⚠️ ？ The returned total = actual count of Patient. This may or may not be correct. Some servers ignore search parameters that they do not understand, which results in a get all query. 


SMILE-CDR
---------
Error with request. Status: 405. Caused by: {'issue': [{'code': 'processing',
            'diagnostics': ':missing modifier is disabled on this server',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET https://try.smilecdr.com:8000/Patient
❌ Found None Patient matching these params {'_total': 'accurate', 'gender:missing': True}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Patient
✅ Found 1776 Patient matching these params {'_total': 'accurate', 'ge

## Comparison Operators

These represent comparators like >, <, =, <=, >=, etc.

https://www.hl7.org/fhir/search.html#prefix

In [7]:
queries = [
    {
        'desc': 'Get all glucose in blood observations with value > 5 mmol/l',
        'endpoint': 'Observation',
        'params': {'value-quantity': 'gt5', 'code': '15074-8'}
    }
]
execute_queries(queries)

***********************************************************
Get all glucose in blood observations with value > 5 mmol/l
***********************************************************
AIDBOX
------
Error with request. Status: 500. Caused by: {'id': 'exception',
 'issue': [{'code': 'exception',
            'diagnostics': 'ERROR: Could not extract as number {"code": '
                           '"mmol/L", "unit": "mmol/l", "value": 3.98, '
                           '"system": "http://unitsofmeasure.org"}',
            'severity': 'fatal'}],
 'resourceType': 'OperationOutcome',
 'text': {'div': 'ERROR: Could not extract as number {"code": "mmol/L", '
                 '"unit": "mmol/l", "value": 3.98, "system": '
                 '"http://unitsofmeasure.org"}',
          'status': 'generated'}}
Success: GET https://kidsfirstr4.aidbox.app/fhir/Observation
❌ Found None Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


SMILE-CDR
---------
Succ

## Searching Coded Things

Tokens are basically coded things - its an attribute with a code, that comes from a system, and has associated
text too. If a SearchParameter is of type `token` this means when used in a query, by default, the token's
code is searched.

For example Specimen.bodysite is a `token` type search parameter. That means
you can search for specimens by bodysite codes like this: /Specimen?bodysite=49852007
    
Also by default (if server supports it) you can search for a token by its text like this:
/Specimen?bodysite:text=<text version representation of 49852007>

https://www.hl7.org/fhir/search.html#token

In [8]:
queries = [
    {
        'desc': 'Get all specimens with code text = Left median cubital vein',
        'endpoint': 'Specimen',
        'params': {'bodysite:text': 'Left median cubital vein'}
    },
    {
        'desc': 'Get all specimens with bodysite code = 49852007 (Left median cubital vein)',            
        'endpoint': 'Specimen',
        'params': {'bodysite': '49852007'}
    },
    {
        'desc': 'Get all anemia conditions by code',            
        'endpoint': 'Condition',
        'params': {'code': '271737000'}
    }
]
execute_queries(queries)

***********************************************************
Get all specimens with code text = Left median cubital vein
***********************************************************
AIDBOX
------
Error with request. Status: 500. Caused by: {'id': 'exception',
 'issue': [{'code': 'exception',
            'diagnostics': 'ERROR: operator does not exist: text[] ~~* '
                           'unknown\n'
                           '  Hint: No operator matches the given name and '
                           'argument types. You might need to add explicit '
                           'type casts.\n'
                           '  Position: 87',
            'severity': 'fatal'}],
 'resourceType': 'OperationOutcome',
 'text': {'div': 'ERROR: operator does not exist: text[] ~~* unknown\n'
                 '  Hint: No operator matches the given name and argument '
                 'types. You might need to add explicit type casts.\n'
                 '  Position: 87',
          'status': 'generate

## Search Using References

Chaining and reverse chaining

https://www.hl7.org/fhir/search.html#chaining

In [9]:
# Get a sample patient name
from pprint import pprint
c = SERVER_CONFIG['azure']
patient, sc = get(f"{c['base_url']}/Patient/PT-00001")
patient_name = patient['name'][0]['given'][0]

queries = [
    {
        'desc': 'Get all specimens for a patient by patient ID',
        'endpoint': 'Specimen',
        'params': {'subject:Patient': "PT-00001"}
    },
    {
        'desc': 'Get all specimens for a patient using their name (chained search parameters)',
        'endpoint': 'Specimen',
        'params': {'subject:Patient.name': patient_name}
    },
    {
        'desc': 'Get patients with a specimen that has body site denoted by <code> (reverse chained parameters)',
        'endpoint': 'Patient',
        'params': {'_has:Specimen:patient:bodysite': '49852007'}
    },
    {
#        This should return all patients - since every observation is a (15074-8) glucose in blood
        'desc': 'Get all the patients that have a Specimen where specimen has an observation.status=final.',
        'endpoint': 'Patient',
        'params': {'_has:Specimen:patient:_has:Observation:code': '15074-8'}
    },
]

execute_queries(queries)

Success: GET https://kids-first-fhir-service.azurewebsites.net/Patient/PT-00001
*********************************************
Get all specimens for a patient by patient ID
*********************************************
AIDBOX
------
Success: GET https://kidsfirstr4.aidbox.app/fhir/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


AZURE
-----
Error with request. Status: 403. Caused by: {'id': '4b423b79-2ee2-4e8c-a99c-25e6f062506e',
 'issue': [{'code': 'forbidden',
            'diagnostics': "Modifier 'Patient' is not supported for search "
                           "parameter 'subject'.",
            'severity':

## Include Referenced Resources

Return nested resources in the payload

https://www.hl7.org/fhir/search.html#revinclude

In [10]:
queries = [
    {
        'desc': 'Get a specimens with its patient',
        'endpoint': 'Specimen',
        'params': {'identifier': "BS-00001-0",
         '_include': 'Specimen:patient'}
    },
    {
        'desc': 'Get an observation and the specimen it is about',
        'endpoint': 'Observation',
        'params': {'identifier': "OB-00001-0",
         '_include': 'Observation:specimen'}
    }
]
execute_queries(queries, display_content=True)


********************************
Get a specimens with its patient
********************************
AIDBOX
------
Success: GET https://kidsfirstr4.aidbox.app/fhir/Specimen
{'entry': [{'fullUrl': 'https://kidsfirstr4.aidbox.app/Patient/PT-00001',
            'resource': {'extension': [{'extension': [{'url': 'text',
                                                       'valueString': 'American '
                                                                      'Indian '
                                                                      'or '
                                                                      'Alaska '
                                                                      'Native'},
                                                      {'url': 'ombCategory',
                                                       'valueCoding': {'code': '1002-5',
                                                                       'display': 'American '
                          

Error with request. Status: 403. Caused by: {'id': '9b492cc2-4dc8-45c6-a23d-5f3ee54b3e1f',
 'issue': [{'code': 'forbidden',
            'diagnostics': 'Include expression is not supported.',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET https://kids-first-fhir-service.azurewebsites.net/Specimen
{'id': '9b492cc2-4dc8-45c6-a23d-5f3ee54b3e1f',
 'issue': [{'code': 'forbidden',
            'diagnostics': 'Include expression is not supported.',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
❌ Found None Specimen matching these params {'_include': 'Specimen:patient',
 '_total': 'accurate',
 'identifier': 'BS-00001-0'}


VONK-KIDSFIRST-PUBLIC
---------------------
Success: GET https://fhir.simplifier.net/KidsFirstTestR4/Specimen
{'entry': [{'fullUrl': 'https://fhir.simplifier.net/KidsFirstTestR4/Specimen/BS-00000-0',
            'resource': {'collection': {'bodySite': {'coding': [{'code': '49852007',
                              

Success: GET https://kidsfirstr4.aidbox.app/fhir/Observation
{'entry': [{'fullUrl': 'https://kidsfirstr4.aidbox.app/Specimen/BS-00001-1',
            'resource': {'collection': {'bodySite': {'coding': [{'code': '49852007',
                                                                 'display': 'Structure '
                                                                            'of '
                                                                            'median '
                                                                            'cubital '
                                                                            'vein '
                                                                            '(body '
                                                                            'structure)',
                                                                 'system': 'http://snomed.info/sct'}],
                                                     'text': 'Left medi

Success: GET http://hapi.fhir.org/baseR4/Observation
{'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/Observation/OB-00001-0',
            'resource': {'code': {'coding': [{'code': '15074-8',
                                              'display': 'Glucose '
                                                         '[Moles/volume] in '
                                                         'Blood',
                                              'system': 'http://loinc.org'}]},
                         'effectivePeriod': {'start': '2013-04-02T09:30:10+01:00'},
                         'id': 'OB-00001-0',
                         'identifier': [{'value': 'OB-00001-0'}],
                         'interpretation': [{'coding': [{'code': 'H',
                                                         'display': 'High',
                                                         'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation'}]}],
                         'issued

## Full Text Search

In [11]:
queries = [
    {
        'desc': 'Full text search - get all specimens w/ code text = Left median cubital vein',
        'endpoint': 'Specimen',
        'params': {'_content': '"Left median cubital vein"'}
    }
]
execute_queries(queries)

****************************************************************************
Full text search - get all specimens w/ code text = Left median cubital vein
****************************************************************************
AIDBOX
------
Success: GET https://kidsfirstr4.aidbox.app/fhir/Specimen
✅ Found 10 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 26 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Specimen
✅ Found 210 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Specimen
✅ Found 20 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}
⚠️ ？ The returned total = actual count of Spec

## Custom Search Parameter

When you add an extension to a resource, you must create a SearchParameter in order to search for resources
by that extension. For example, you've created an `race` extension and use that on Patient resources. Now
you want to do searches like this: /Patient?race=2028-9 or /Patient?race:text=Asian. In order to do that
you will need to create a SearchParameter for the race extension.

The example queries below demonstrate searches with a custom search parameter

In [12]:
queries = [
    {
        'desc': 'Search on extension - get all patients with particular race',
        'endpoint': 'Patient',
        'params': {'race': '2028-9'}
    }
]
execute_queries(queries)

***********************************************************
Search on extension - get all patients with particular race
***********************************************************
AIDBOX
------
Error with request. Status: 500. Caused by: {'id': 'exception',
 'issue': [{'code': 'exception',
            'diagnostics': 'Search for {:resourceType :Patient, :type :param, '
                           ':name "race", :values [{:value "2028-9"}], '
                           ':search-param #object[clojure.lang.Agent '
                           '0x76a2bfd4 {:status :ready, :val nil}]} is not '
                           'implemented',
            'severity': 'fatal'}],
 'resourceType': 'OperationOutcome',
 'text': {'div': 'Search for {:resourceType :Patient, :type :param, :name '
                 '"race", :values [{:value "2028-9"}], :search-param '
                 '#object[clojure.lang.Agent 0x76a2bfd4 {:status :ready, :val '
                 'nil}]} is not implemented',
          'status': '

## Custom Search Query

Sometimes the RESTful FHIR search API cannot satisfy your query needs. It would be nice if you could query the underlying database(s) of the FHIR server directly. 

Aidbox is the only server solution that seems to support this via their API

In [13]:
# Only Aidbox supports this
from requests.auth import HTTPBasicAuth

c = SERVER_CONFIG['aidbox']
sql_str = (
    """
    SELECT
    p.id AS patient_id,
    s.id AS specimen_id,
    s.resource AS specimen    
    FROM 
    patient AS p
    JOIN specimen AS s
    ON p.id = s.resource->'subject'->>'id';
    """
)
url = f"{c['base_url'].rstrip('/fhir')}/$sql"
resp = requests.post(
    url,
    auth=HTTPBasicAuth(c['username'], c['password']),
    data=sql_str,
    headers={'Content-Type': 'text/yaml'}
)
pprint(resp.json())

[{'patient_id': 'PT-00000',
  'specimen': {'collection': {'bodySite': {'coding': [{'code': '49852007',
                                                       'display': 'Structure '
                                                                  'of median '
                                                                  'cubital '
                                                                  'vein (body '
                                                                  'structure)',
                                                       'system': 'http://snomed.info/sct'}],
                                           'text': 'Left median cubital vein'},
                              'method': {'coding': [{'code': 'LNV',
                                                     'system': 'http://terminology.hl7.org/CodeSystem/v2-0488'}]}},
               'extension': [{'url': 'http://fhirr4.kids-first.io/fhir/StructureDefinition/specimen-analyte-type',
                              

                              'method': {'coding': [{'code': 'LNV',
                                                     'system': 'http://terminology.hl7.org/CodeSystem/v2-0488'}]}},
               'extension': [{'url': 'http://fhirr4.kids-first.io/fhir/StructureDefinition/specimen-analyte-type',
                              'valueString': 'RNA'}],
               'identifier': [{'value': 'BS-00007-1'}],
               'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/Specimen']},
               'subject': {'id': 'PT-00007', 'resourceType': 'Patient'}},
  'specimen_id': 'BS-00007-1'},
 {'patient_id': 'PT-00008',
  'specimen': {'collection': {'bodySite': {'coding': [{'code': '49852007',
                                                       'display': 'Structure '
                                                                  'of median '
                                                                  'cubital '
                                                         

## Exact Path Match

This is only supported by Aidbox. Sometimes you want to search by an attribute in your resource payload, 
but you don't have a SearchParameter for it defined and loaded into the server. 

With aidbox, you can simply search by the path to that attribute in the resource payload. For example, I want to search for Specimens by the analyte type extension but I don't have a SearchParameter defined for it. I can search for Specimens by analyte type like this:

In [14]:
# Only Aidbox supports this
queries = [
    {
        'desc': 'Get all specimens with analyte type = DNA',
        'endpoint': 'Specimen',
        'params': {'.extension.0.valueString': 'DNA'}
    }
]
execute_queries(queries)

*****************************************
Get all specimens with analyte type = DNA
*****************************************
AIDBOX
------
Success: GET https://kidsfirstr4.aidbox.app/fhir/Specimen
✅ Found 13 Specimen matching these params {'.extension.0.valueString': 'DNA', '_total': 'accurate'}


SMILE-CDR
---------
Error with request. Status: 400. Caused by: {'issue': [{'code': 'processing',
            'diagnostics': 'Unknown search parameter "". Value search '
                           'parameters for this search are: [_id, _language, '
                           'accession, bodysite, collected, collector, '
                           'container, container-id, identifier, parent, '
                           'patient, status, subject, type]',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET https://try.smilecdr.com:8000/Specimen
❌ Found None Specimen matching these params {'.extension.0.valueString': 'DNA', '_total': 'accurate'}


HAPI
----
Err